# Adding a problem to pySDC

In this notebook, we will implement a simple advection-diffusion equation $$u_t = f(u) = \nu u_{xx} + cu_x$$ with periodic boundary conditions using finite differences.
Let's have a look at the math that we need to implement in order to integrate this with SDC.

The SDC iteration is $$(1-\Delta t \tilde{q}_{m+1,m+1} f)(u_{m+1}^{k+1}) = u_0 + \Delta t \sum_{j=1}^m \tilde{q}_{m+1, j} f(u_{j}^{k+1}) -\Delta t \sum_{j=1}^{m+1}\tilde{q}_{m+1,j} f(u_{j}^{k}) + \Delta t \sum_{j=1}^M q_{m+1, j} (f)(u_{j}^k), \quad m = 0, \dots, M-1,$$ where $q_{ij}$ and $\tilde{q}_{ij}$ are the entries of the quadrature matrix and preconditioner respectively.
The right hand side is a weighted sum of evaluations of $f$ with solutions at various collocation nodes and iterations and the left hand side is essentially an implicit Euler step with step size modified by the preconditioner.
Therefore, the only math we need to implement in the problem class is **evaluating $f$** and **solving implicit Euler steps**.

Keep in mind that you can use an explicit preconditioner, in which case you don't even need the implicit Euler solves, but explicit SDC is rarely useful.
If implementing an implicit Euler step for your problem is too difficult, consider splitting off difficult terms and integrating them explicitly or designing a novel SDC scheme.
The iterative nature of SDC allows a lot of freedom which is one of the key advantages of SDC.

## Evaluating $f$

In finite difference discretizations, we compute derivatives by multiplying matrices with a vector of solution values at the grid points.
In pySDC we have some infrastructure for generating finite difference matrices, making this very easy to implement.

We generate one matrix for the advection part, i.e. we discretize the first derivative, and one matrix for the diffusion part via the second derivative.
The $f$ evaluation is then a weighted sum of the matrices multiplied by the solution vector.

In [1]:
import numpy as np
from pySDC.helpers.problem_helper import get_finite_difference_matrix, get_1d_grid

# problem parameters
nu = 1e-3
c = 1e-1
N = 128

# setup grid
dx, grid = get_1d_grid(size=N, bc='periodic', left_boundary=0, right_boundary=2*np.pi)

# setup finite difference matrices
fd_params = {'order': 4, 'stencil_type': 'center', 'dx': dx, 'size': N, 'dim': 1, 'bc': 'periodic'}
advection, _ = get_finite_difference_matrix(derivative=1, **fd_params)
diffusion, _ = get_finite_difference_matrix(derivative=2, **fd_params)

def eval_f(u):
    return (c*advection + nu*diffusion) @ u

Before moving on, we briefly verify that we did the correct thing by comparing the numerical derivatives to exact ones for a simple sine wave:

In [2]:
u = np.sin(grid)
assert np.allclose(diffusion@u, -np.sin(grid))  # check diffusion part
assert np.allclose(advection@u, np.cos(grid))  # check advection part
assert np.allclose(eval_f(u), -nu*np.sin(grid)+c*np.cos(grid))  # check f evaluation

## Implicit Euler step

In the implicit Euler step, we solve $(1-\Delta t f)(u) = y$, where $y$ is some arbitrary right hand side. We use a simple direct solver and the matrices we used to evaluate $f$ here:

In [3]:
from scipy.sparse.linalg import spsolve
from scipy.sparse import eye

def implicit_euler(rhs, dt):
    A = eye(N) - dt*(c*advection + nu*diffusion)
    return spsolve(A, rhs)

Let's once again verify the implementation. This time, we will compare to exact solutions for either the diffusion or the advection equation, which we obtain by setting the $c$ or $\nu$ parameters to 0.

In [4]:
u = np.sin(grid)
dt = 1e-2

# check advection
nu = 0
c = 1e-1
assert np.allclose(implicit_euler(u, dt), np.sin(grid+c*dt))

# check diffusion
nu = 1e-1
c = 0
assert np.allclose(implicit_euler(u, dt), np.sin(grid)*np.exp(-nu*dt))

## Implementing this class in pySDC

So far, we have looked at what math you need to implement in order to do SDC with your problem, now we will set up a class with the same functionality that we can then hand to pySDC to actually run SDC.

In [5]:
from pySDC.core.problem import Problem, WorkCounter
from pySDC.implementations.datatype_classes.mesh import mesh
import scipy.sparse as sp
import numpy as np

class AdvectionDiffusion(Problem):
    dtype_u = mesh  # wraps numpy ndarray
    dtype_f = mesh
    
    def __init__(self, N, c, nu):
        init = (N, None, np.dtype('float64'))  # describes how to initialize data
        super().__init__(init=init)
        
        # setup grid
        dx, self.grid = get_1d_grid(size=N, bc='periodic', left_boundary=0, right_boundary=2*np.pi)

        # setup finite difference matrices
        fd_params = {'order': 4, 'stencil_type': 'center', 'dx': dx, 'size': N, 'dim': 1, 'bc': 'periodic'}
        advection, _ = get_finite_difference_matrix(derivative=1, **fd_params)
        diffusion, _ = get_finite_difference_matrix(derivative=2, **fd_params)

        # setup matrices used in eval_f and implicit_euler
        self.A = c * advection + nu * diffusion
        self.Id = sp.eye(N)

        # store attribute and register them as parameters
        self._makeAttributeAndRegister('N', 'c', 'nu', localVars=locals(), readOnly=True)
        self.work_counters['gmres'] = WorkCounter()

    def eval_f(self, u, t):
        me = self.f_init
        me[...] = self.A @ u
        return me

    # the implicit Euler step is called solve_system here, because it can be sth. else in a different SDC scheme
    def solve_system(self, rhs, factor, u0, t):
        me = self.u_init
        me[...], _ = sp.linalg.gmres(self.Id - factor*self.A, rhs, x0=u0, callback=self.work_counters['gmres'], rtol=1e-10, callback_type='pr_norm')
        return me

    def u0(self):
        me = self.u_init
        me[...] = np.sin(self.grid)
        return me

Let's do the same tests once more:

In [6]:
prob = AdvectionDiffusion(128, 1e-1, 1e-3)
u = prob.u0()
assert np.allclose(prob.eval_f(u, 0), -prob.nu*np.sin(prob.grid)+prob.c*np.cos(prob.grid))

dt = 1e-2
advection_prob = AdvectionDiffusion(N=128, c=1e-1, nu=0)
u = advection_prob.u0()
assert abs(advection_prob.solve_system(u, dt, u, 0)- np.sin(advection_prob.grid+advection_prob.c*dt)) < 1e-6

diffusion_prob = AdvectionDiffusion(N=128, c=0, nu=1e-1)
u = diffusion_prob.u0()
assert abs(diffusion_prob.solve_system(u, dt, u, 0)- np.sin(diffusion_prob.grid)*np.exp(-diffusion_prob.nu*dt)) < 1e-6